<a href="https://colab.research.google.com/github/KelvinLam05/profanity_detection/blob/main/profanity_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Goal of the project**

Countless websites today accept content from users in the form of text. Content such as comments on posts on social media platforms, reviews on products on e-commerce websites and articles on platforms like Medium, is increasing everyday at a rate faster than before. With such volume of content being generated, there’s an increasing load on moderators to moderate this content. Automating a few aspects of this moderation process would not only aid the moderators in the moderation process, but will also make the entire process more efficient. Detecting profanity in content is one such use-case.

**Load the packages**

In [ ]:
import pandas as pd
import numpy as np
import fasttext

**Load the data**

For this project I’ve used the [Jigsaw toxic comment classification dataset](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) from Kaggle. We are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are: toxic, severe_toxic, obscene, threat, insult, and identity_hate. 

In [ ]:
# Load dataset
df = pd.read_csv('/content/train.csv').iloc[: , 1:]

In [ ]:
# Examine the data
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
# Overview of all variables, their datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   comment_text   159571 non-null  object
 1   toxic          159571 non-null  int64 
 2   severe_toxic   159571 non-null  int64 
 3   obscene        159571 non-null  int64 
 4   threat         159571 non-null  int64 
 5   insult         159571 non-null  int64 
 6   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(1)
memory usage: 8.5+ MB


**Define the target variable**

Let’s first restructure this dataset to contain only one label type which will signify whether the sentence is profane or not. We will consider the comment to be profane if any of toxic, severe_toxic, obscene, threat, insult, identity_hate are marked as 1.

In [ ]:
cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
df['label'] = df[cols].any(axis = 1).astype(int)

In [ ]:
df = df[['comment_text', 'label']]

**Examine the class imbalance**

By running value_counts( ) on the label column containing our target variable, we can see that the data are imbalanced.

In [ ]:
df['label'].value_counts()

0    143346
1     16225
Name: label, dtype: int64

**Preprocessing the data**

We will need to preprocess our text to remove misleading junk and noise in order to get the best results from our model.

In [ ]:
import re
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

In [ ]:
import unicodedata
from emoji import demojize

In [ ]:
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
def text_cleaning(text_data):

  # Remove accented characters
  text_data = unicodedata.normalize('NFKD', text_data).encode('ascii', 'ignore').decode('utf-8', 'ignore')

  # Case conversion
  text_data = text_data.lower()

  # Demojize
  text_data = demojize(text_data)

  # Reducing repeated punctuations
  pattern_punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
  text_data = pattern_punct.sub(r'\1', text_data)
  
  # Prevent redundant replacements of single-space with single-space
  text_data = re.sub(' {2,}',' ', text_data)
  
  # Remove special characters
  text_data = re.sub(r"[^a-zA-Z?!]+", ' ', text_data)

  # Converting text to strings
  text_data = str(text_data)

  # Tokenization
  tokenizer = ToktokTokenizer()
  text_data = tokenizer.tokenize(text_data)

  # Removing stopwords
  text_data = [item for item in text_data if item not in stop_words]
  
  # Lemmatization
  text_data = [lemmatizer.lemmatize(word = w, pos = 'v') for w in text_data]
  
  # Convert list of tokens to string data type
  text_data = ' '.join (text_data)

  return text_data

In [ ]:
df['comment_text'] = df['comment_text'].apply(text_cleaning)

**Split the training and test data**

Before training our classifier, we need to split the data into train and test. We will use the test set to evaluate how good the learned classifier is on new data.

In [ ]:
X = df.drop(['label'], axis = 1)

In [ ]:
y = df['label']

For the train-test split we use stratify to keep the ratio of category labels.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

In [ ]:
X_train = pd.DataFrame(X_train)

In [ ]:
train = X_train.join(y_train)

In [ ]:
X_test = pd.DataFrame(X_test)

In [ ]:
test = X_test.join(y_test)

**Formatting the labels**

The format of the text that goes into a fastText is a series/list, with each element as a string of text including its respective labels. All the labels/categories in fastText start by the "`__label__`" prefix, which is how fastText recognize what is a label or what is a word. The model is then trained to predict the labels given the word in the document. So now we will add `__label__` in front of the category for fastText to read it as a label and then combine the labels and words into a single string.

In [ ]:
train['label'] = '__label__' + train['label'].astype(str)

In [ ]:
test['label'] = '__label__' + test['label'].astype(str)

In [ ]:
train['label'] = train['label'] + ' ' + train['comment_text']

In [ ]:
test['label'] = test['label'] + ' ' + test['comment_text']

In [ ]:
train.head()

,comment_text,label
94175,message thank message fun ! feb utc,__label__0 message thank message fun ! feb utc
50277,fallout shelter remove two part unsourced stat...,__label__0 fallout shelter remove two part uns...
62862,hair loss information spam link relate topic,__label__0 hair loss information spam link rel...
35721,thank honestly feel hour block bite much quest...,__label__0 thank honestly feel hour block bite...
121873,warn support propaganda ? nice try henrik try ...,__label__0 warn support propaganda ? nice try ...


In [ ]:
test.head()

,comment_text,label
63958,thank help plot upon time west one time favori...,__label__0 thank help plot upon time west one ...
11723,nothing ambiguous logic evolution guy upgrade ...,__label__0 nothing ambiguous logic evolution g...
18282,go hell fatso hey dickwad ! ignore poke hole r...,__label__1 go hell fatso hey dickwad ! ignore ...
145258,add justice development party turkey akp add l...,__label__0 add justice development party turke...
152834,hi add link incidents think constructive pleas...,__label__0 hi add link incidents think constru...


**Saving DataFrames as text files**

In the following step, we are saving our DataFrames as text files.

In [ ]:
import csv

In [ ]:
# Write test and train into files
train.to_csv('train.txt', 
             index = False, 
             sep = ' ',
             header = None, 
             quoting = csv.QUOTE_NONE, 
             quotechar = "", 
             escapechar = " ")

test.to_csv('test.txt', 
            index = False, 
            sep = ' ',
            header = None, 
            quoting = csv.QUOTE_NONE, 
            quotechar = "", 
            escapechar = " ")

**Our classifier**

We are now ready to train our classifier:

In [ ]:
model = fasttext.train_supervised('train.txt')

Now let's see how the model does on the test set.

In [ ]:
model.test('test.txt')  

(31915, 0.9606767977440075, 0.9606767977440075)

The output are the number of samples (here 31915), the precision at one (0.96) and the recall at one (0.96).